# Explore model performances

This document presents an overview of the results from a variety of model outputs, including further modelling chip probabilities.

There are several aspects of the predictions we'd like to explore and compare across models:

- Overall metrics:
     - [ ] Accuracy ([micro-F1](https://towardsdatascience.com/multi-class-metrics-made-simple-part-ii-the-f1-score-ebe8b2c2ca1): general over-simplified measure of how many chips the model got right
     - [ ] [Kappa score](https://towardsdatascience.com/multi-class-metrics-made-simple-the-kappa-score-aka-cohens-kappa-coefficient-bdea137af09c): measure of how many chips the model got right, _above_ what it would have got by pure chance
- Class-based metrics:
     - [ ] Accuracy: proportion of chips the model got right for each class
     - [ ] Macro-F1: overview of how good the model is at predicting each single class
- Confusion matrices: more disaggregated view on predictions. Detailed but not summarising.
- Spatial metrics: compare whether the distribution of predicted labels over space resembles that of the true values
     - [ ] Joint count statistics